### Using LangExtract to extract entities and their relations

[LangExtract](https://github.com/google/langextract/) is a Python library that uses LLMs to extract structured information from unstructured text documents based on user-defined instructions, in particular, a user-defined prompt and a few examples illustrating the kind of information that is to be extracted.

Like the previous information extraction (IE) approach we explored (in week 7), this one is also based on the user's instructions (prompt). However, unlike the previous IE approach that was based on (annotated) data schema, this approach relies on an example (one or more) of the data to be extracted.

We will explore LangExtract's information extraction approach on an example of extracting information about tech companies, their software products and their key employees, from the text of a news article.

In [ ]:
import textwrap
import langextract as lx
import requests
from bs4 import BeautifulSoup

from collections import defaultdict
from pathlib import Path

Step 1: Define a concise prompt with instructions what information should be extracted

In [ ]:
prompt = textwrap.dedent("""
    You are highly experienced in extracting named entities and their relations from text, especially from news articles and similar kinds of textual content.

    Your task is to extract people, companies, and software products mentioned in the text given below.

    Provide meaningful attributes for each entity you identify, to establish connections between entities of different types. For example, to establish a connection between a person and a company or a company and a software product.

    Important: Use exact text from the input for extraction text. Do not paraphrase.
    Extract entities in order of their appearance with no overlapping text spans.
""")

Step 2: Provide a high-quality example to guide the model

In [ ]:
examples = [
    lx.data.ExampleData(
        text=(
            "Llion Jones, a co-founder of Sakana AI, has recently presented one of the company's key products: an AI-based tool called AI Scientist."
        ),
        extractions=[
            lx.data.Extraction(
                extraction_class="person",
                extraction_text="Llion Jones",
                attributes={"works_for": "Sakana AI", "role": "co-founder"},
            ),
            lx.data.Extraction(
                extraction_class="company",
                extraction_text="Sakana AI",
                attributes={"co-founder": "Llion Jones", "product":"AI Scientist"},
            ),
            lx.data.Extraction(
                extraction_class="software_product",
                extraction_text="AI Scientist",
                attributes={"developed_by": "Sakana AI"},
            ),
        ],
    )
]

Step 3: Prepare text to be used for IE

We will try out the LangExtract's information extraction approach on a [techcrunch article](https://techcrunch.com/2025/08/03/inside-openais-quest-to-make-ai-do-anything-for-you/) about LLM development inside OpenAI.
To that end, we need two additional python libraries:
* requests - for pulling the content of the article from the given URL
* beautifulsoup - for extracting the main text of the article

In [ ]:
content = []
url = "https://techcrunch.com/2025/08/03/inside-openais-quest-to-make-ai-do-anything-for-you/"

try:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    content_elem = soup.find(lambda elem: (elem.name=='div') and elem.has_attr('class') and (elem['class'][0] == 'entry-content'))
    for p in content_elem.findAllNext(name='p'):
        if p.text:
            content.append(p.text)
except Exception as e:
    print(e)

input_text = "\n".join(content)

# store the content in a local file so that we do not need to pull it from the web each time
with open(Path.cwd() / 'data' / 'tech_crunch_article.txt', 'w') as f:
    f.write(input_text)

# input_text[:200]

Step 4: Run the extraction on the target text

In [ ]:
result = lx.extract(
    text_or_documents=input_text,
    prompt_description=prompt,
    examples=examples,
    model_url="http://localhost:11434", # Automatically selects Ollama provider
    model_id="gemma3:4b",
    fence_output=False,
    use_schema_constraints=False,
    debug=False,
    # extraction_passes=2 # Number of sequential extraction attempts to improve recall by finding additional entities
)

Note: in the above call of the `extract` function, we set `fence_output=False` and `use_schema_constraints=False` because LangExtract doesn't (yet) implement schema constraints for models other than Gemini. That is, only for Gemini models, it can be requested that the produced output is fully aligned wit the given schema.

For more details about the `extract` function, check the source code, available [here](https://github.com/google/langextract/blob/main/langextract/extraction.py)

We will now explore the results.

The results are stored in an instance of [`AnnotatedDocument` class](https://github.com/google/langextract/blob/main/langextract/core/data.py#L184), and the extracted data is in its `extractions` list attribute:

In [ ]:
len(result.extractions)

In [ ]:
print(result.extractions[0])

In [ ]:
people = defaultdict(list)
companies = defaultdict(list)
products = defaultdict(list)
other = defaultdict(list)

for extr_item in result.extractions:

    ext_type = extr_item.extraction_class
    ext_lbl = extr_item.extraction_text

    if ext_type == "person":
        people[ext_lbl.lower()].append(extr_item)
    elif ext_type == "company":
        companies[ext_lbl.lower()].append(extr_item)
    elif ext_type  == 'software_product':
        products[ext_lbl.lower()].append(extr_item)
    else: other[ext_lbl.lower()].append(extr_item)

print(f"Number of unique persons: {len(people)}")
print(f"Number of unique companies: {len(companies)}")
print(f"Number of unique software products: {len(products)}")
print(f"Number of other unique entities: {len(other)}")

In [ ]:
list(other.items())[0]

In [ ]:
def print_extracted_entities(entity_type:str, entities:dict):
    print(f"Extracted entities of type {entity_type.upper()}:")
    for entity, entity_occurrences in entities.items():
        for eo in entity_occurrences:
            position_info = ""
            if eo.char_interval:
                start, end = eo.char_interval.start_pos, eo.char_interval.end_pos
                position_info = f" (pos: {start}-{end})"
            print(f"• {eo.extraction_text} {position_info}: {eo.attributes if eo.attributes else 'no attr.'}")

In [ ]:
# print_extracted_entities(dict(people))
print_extracted_entities("person", people)

In [ ]:
print_extracted_entities("company", companies)

In [ ]:
print_extracted_entities("software product", products)

The results can be store in the JSONL (JSON Lines) format, a text-based format where each line represents a valid JSON object; it is used for storing structured data records. For more info, see [https://jsonltools.com/what-is-jsonl](https://jsonltools.com/what-is-jsonl)

In [ ]:
results_fname = "tech_companies_IE_results"

lx.io.save_annotated_documents([result],
                               output_name=f"{results_fname}.jsonl",
                               output_dir=Path.cwd() / 'ie_results')

The JSONL file can be used for generating an HTML document with interactive visualization of the results

In [ ]:
results_json = f"{Path.cwd()}/ie_results/{results_fname}.jsonl"

html_content = lx.visualize(results_json)

with open(f"{Path.cwd()}/ie_results/{results_fname}.html", "w") as f:
     f.write(html_content.data)

Identify relationships between entities, as they can offer a solid ground for creating a knowledge graph

In [ ]:
relations = defaultdict(list)

for extr_item in result.extractions:

    ext_lbl = extr_item.extraction_text
    if extr_item.attributes:
        for ext_attr in extr_item.attributes.items():
            attr_rel, attr_val = ext_attr
            relations[attr_rel].append((ext_lbl, attr_val))


for rel in relations.keys():
    print(rel)
    for pair in relations[rel]:
        print(pair)
    print("-------------------")

In [ ]:
# list the distinct kinds of relations
relations.keys()

Some of these relations can / should be merged (e.g., 'works_for' and 'works_at', or 'employee' and 'employs'), to better capture the semantics of relations and set the grounds for knowledge graph creation. [This YouTube video](https://www.youtube.com/watch?v=dPL2vRDunMw) shows how that can be done.

#### Explore LangExtract further

To get a better understanding of LangExtract features and how it can be used for information extraction, it is recommended to explore:
* the examples available at the LangExtract's GitHub repo and especially the [Medication extraction example](https://github.com/google/langextract/blob/main/docs/examples/medication_examples.md)
* the examples presented in the [DataCamp's LangExtract tutorial](https://www.datacamp.com/tutorial/langextract)

#### Compare and contract LangExtract w/ alternative approaches to information extraction

A very nice comparison is given in the aforementioned [DataCamp's tutorial](https://www.datacamp.com/tutorial/langextract), specifically in the table towards the end of the article